# 4. Inference

In [1]:
import sys, os
sys.path.append("../../..")

import pathSetting
import pandas as pd
from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
from KETIToolDL.CLUSTTool.common import p2_dataSelection as p2
from KETIToolDL.CLUSTTool.common import p4_testing as p4
from KETIToolDL.CLUSTTool.Regression import p4_testing as p4R
from KETIToolDL.CLUSTTool.RNNPrediction import p5_inference as p5

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 4-1. (just for test) Data Preparation 

In [8]:
# 0. pick only one data
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())
# dataX

dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]
dataRoot = 'ml_data_integration'

dataX = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataRoot)[:144]
datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataRoot)[:1]
from KETIPreDataTransformation.dataFormatTransformation.DFToNPArray import transDFtoNP
inputX, inputy = transDFtoNP(dataX, datay)
print(inputX, inputy) # Real Input

/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest
/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest
[[[40.26       40.26       40.29       ... 42.09       42.03
   41.9       ]
  [43.2        43.16333333 42.96333333 ... 42.36333333 42.43333333
   42.5       ]
  [38.53       38.5        38.63333333 ... 38.29       38.36333333
   38.29      ]
  ...
  [ 4.2         4.          3.8        ...  8.3         8.33333333
    8.36666667]
  [40.         40.         40.         ... 40.         40.
   40.        ]
  [ 4.          3.83333333  3.66666667 ...  7.          6.66666667
    6.33333333]]] [17.37000084]


/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:31: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:32: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]


In [3]:
# 1 Read Parameter and model information
ModelMeta =p1.readJsonData(pathSetting.trainModelMetaFilePath)
modelList = list(ModelMeta.keys())
modelName = modelList[1]

X_scalerFilePath = ModelMeta[modelName]["X_scalerFilePath"]
y_scalerFilePath = ModelMeta[modelName]["y_scalerFilePath"]
featureList = ModelMeta[modelName]["featureList"]
target = ModelMeta[modelName]["target"]
scalerParam = ModelMeta[modelName]["scalerParam"]
model_method = ModelMeta[modelName]["model_method"]
trainParameter = ModelMeta[modelName]["trainParameter"]
modelFilePath = ModelMeta[modelName]["modelFilePath"]

In [4]:
#2
from KETIPreDataTransformation.dataFormatTransformation.NPArrayToDF import transNPtoDF
inputXDF, y = transNPtoDF(inputX, inputy,'2000-1-1')

In [5]:
# 3. Scaling
test_x, scaler_X = p4.getScaledTestData(dataX[featureList], X_scalerFilePath, scalerParam)
test_y, scaler_y = p4.getScaledTestData(datay[target], y_scalerFilePath, scalerParam)
# y  값은 더미 값이 들어가는 것임

In [6]:
# 4. Testing
batch_size=1
df_result, result_metrics = p4R.getResultMetrics(test_x, test_y, model_method, target, modelFilePath, scalerParam, scaler_y, trainParameter, batch_size, device)


Start testing data

** Performance of test dataset ==> MSE = 0.03197894245386124, MAE = 0.17882657051086426
** Dimension of result for test dataset = (1,)
scale


/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:31: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:32: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]
/home/keti/.conda/envs/torch/lib/python3.8/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [9]:
df_result

,value,prediction
0,17.370001,13.625372
